<h1>Singledispatchmethod</h1>

<h2>Overloading dei metodi nelle classi</h2>

<p>Come in <a href="http://localhost:8888/notebooks/Jupyter/Python%20Learn/Librerie%20builtin/Decoratori/1.0%20-%20Decoratori%20per%20funzioni%20-%20[functools]Singedispatch%20(overloading%20di%20funzioni).ipynb"><em>@singledispatch</em></a> il dispatcher <U>è il type del primo argomento</U>, in questo caso pero <U>dopo il self o il cls</U> (si perchè puo essere usato anche in un <code class="inline"><em>@classmethod</em></code>).</p>

<p>Andiamo a richiamare la libreria, creiamo una classe e il metodo da decorare che funge da fallback, con i suoi dispatch</p>

In [1]:
from functools import singledispatchmethod

In [2]:
class Negatore:
    @singledispatchmethod
    def neg(self, arg):
        raise NotImplementedError("Impossibile da negare")

    @neg.register
    def _(self, arg: int):
        return -arg

    @neg.register
    def _(self, arg: bool):
        return not arg

In [3]:
pippo = Negatore()
print(pippo.neg(5))

-5


<p>Notare che il primo argomento deve essere sempre il self. Possiamo però usarla anche con le <code class="inline"><em>@classmethod</em></code>, ma anche le <code class="inline"><em>@staticmethod</em></code> e le <code class="inline"><em>@abstractmethod</em></code>. Per fare ciò dobbiamo annidare (nesting) i due decoratori, facendo attenzione all'ordine in cui lo facciamo:</p>

In [4]:
class Negatore:
    @singledispatchmethod
    @classmethod
    def neg(cls, arg):
        raise NotImplementedError("Cannot negate a")

    @neg.register
    @classmethod
    def _(cls, arg: int):
        return -arg

    @neg.register
    @classmethod
    def _(cls, arg: bool):
        return not arg

In [5]:
print(Negatore.neg(5))

-5


<p>Quindi prima il decoratore del singledispatch, poi quello del metodo, come il classmethod in questo caso. Ovviamente adesso invece del self dobbiamo mettere cls.</p>

<h2>Caso particolare: Overloading nell' Overriding dei dunder methods</h2>

<p>In questo caso andiamo a lavorare sul dunder <code class="inline">__add__</code> in una classe</p>

In [6]:
from dataclasses import dataclass
from functools import singledispatchmethod

<p>Una classe di tipo data che prende una nota in forma di stringa, inoltre ne ottiene il suo corrispettivo valore valore numerico (da noi scelto) enumerando una tupla a partire da 1</p>

<p>Quello che si vuole ottenere è che quando andiamo a sommare due oggetti della stessa classe, oppure un oggetto della classe con un intero, dia il valore numerico sommato. Se è maggiore di 12 si ritorna da capo a 1</p>

In [7]:
@dataclass
class Nota:
    let: str

    def __post_init__(self):
        diesis = ('A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#')
        bemolle = ('A', 'Bb', 'B', 'C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab')
        pippo = diesis if self.let in diesis else bemolle
        for n, l in enumerate(pippo, 1):
            if l == self.let:
                self.num = n
    
    @singledispatchmethod
    def __add__(self, other):
        try:
            sum = self.num + other.num
        except:
            raise NotImplementedError(f"Il tipo non è supportato")
        else:
            sum = result - 12 if sum > 12 else sum
            return sum

    @__add__.register
    def _(self, n: int):
        sum = self.num + n
        sum = sum - 12 if sum > 12 else sum
        return sum

In [8]:
do = Nota("C")
la = Nota("A")

print(do+la) # sarebbe 4 + 1
print(do + 9) # sarebbe 13, e quindi ritorna a 1

5
1


<p>Da notare che nella funzione base (quella di fallback) come argomenti abbiamo messo quelle di default del dunder <code class="inline">__add__</code>, ovvero self e other. Nel dispatcher abbiamo messo il self e poi l'argomento che dispatcha.</p>

<p>Per maggiori dettagli la documentazione ufficiale di <a href="https://docs.python.org/3/library/functools.html">functools</a>.
</p>